In [5]:
import requests
import pandas as pd
import matplotlib.pyplot as plt


headers = {'User-Agent': "erickagnyc@gmail.com"}

# get all companies data 
#mapping cik number on to ticker and company name to actually understand what company you're looking at
companyTickers = requests.get(
    "https://www.sec.gov/files/company_tickers.json",
    headers=headers
    )

companyData = pd.DataFrame.from_dict(companyTickers.json(),
                                     orient='index')

# add leading zeros to CIK
companyData['cik_str'] = companyData['cik_str'].astype(
                           str).str.zfill(10)

## Input Ticker to Create CIK

In [12]:
ticker = input('Ticker: ').upper()
cik = companyData[companyData['ticker'] == ticker].cik_str[0]

# get company specific filing metadata
filingMetadata = requests.get(
    f'https://data.sec.gov/submissions/CIK{cik}.json',
    headers=headers
    )

company_info = companyData[companyData['ticker'] == ticker]

company_info

Ticker: ea


,cik_str,ticker,title
367,0000712515,EA,ELECTRONIC ARTS INC.


# Financial Data

In [14]:
terms = [
    'NetIncomeLoss',
    'InterestExpense',
    'IncomeTaxExpenseBenefit',
    'Depreciation',
    'AmortizationOfIntangibleAssets',
    'Revenues',
    'AssetsCurrent',
    'LiabilitiesCurrent',
    'Liabilities',
    'StockholdersEquity',
    'EarningsPerShareDiluted'
]

EA_revenue_data = {
    'fy': [2024, 2023, 2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009],
    'Revenue': [
        7562000000, 7426000000, 6991000000, 5629000000, 5537000000, 4950000000, 
        5150000000, 4845000000, 4396000000, 4515000000, 3575000000, 3797000000, 
        4143000000, 3589000000, 3654000000, 4212000000
    ]
}

# Convert EA revenue data to a DataFrame for easy lookup (changing 'Year' to 'fy')
EArevenue_df = pd.DataFrame(EA_revenue_data)

def get_financial_data(cik, terms, headers):
    # Initialize an empty dictionary to store data for each term
    data_dict = {}

    # Loop through each term
    for term in terms:
        # Request data from the API
        response = requests.get(
            f'https://data.sec.gov/api/xbrl/companyconcept/CIK{cik}/us-gaap/{term}.json',
            headers=headers
        )

        # Load the returned data into a DataFrame
        
        if term == 'EarningsPerShareDiluted': 
            data = pd.DataFrame.from_dict(response.json()['units']['USD/shares'])
        else: 
            data = pd.DataFrame.from_dict(response.json()['units']['USD'])

        # Filter the data for 10-K filings
        data10K = data[data['form'] == '10-K']
        data10K = data10K.reset_index(drop=True)

        # Clean the DataFrame (convert filed dates to datetime)
        data10K["filed"] = pd.to_datetime(data10K["filed"])

        # Keep only the latest 10-K filing for each fiscal year (fy)
        data10K = data10K.sort_values("filed").drop_duplicates(subset=["fy"], keep="last")

        # Store the data in the dictionary with the term as the key
        data_dict[term] = data10K.set_index('fy')['val']

    # After collecting all data, check if the 'Revenues' data has NaN values and replace them
    
    combined_df = pd.DataFrame(data_dict)
    
    if cik == '0000712515':
        revenues = combined_df['Revenues']
        
        # Loop through the fiscal years to find missing values
        for year in revenues.index:
            if pd.isna(revenues.loc[year]):  # If the value is NaN
                # Replace NaN values with the revenue from the provided EA data
                if year in EArevenue_df['fy'].values:
                    revenue_value = EArevenue_df.loc[EArevenue_df['fy'] == year, 'Revenue'].values[0]
                    combined_df.at[year, 'Revenues'] = revenue_value
        
    combined_df = combined_df.fillna(0)
                   
    combined_df["EBITDA"] = (
        combined_df["NetIncomeLoss"] +
        combined_df["InterestExpense"] +
        combined_df["IncomeTaxExpenseBenefit"] +
        combined_df["Depreciation"] +
        combined_df["AmortizationOfIntangibleAssets"]
    )

    combined_df['ROE'] = (
        combined_df["NetIncomeLoss"] / combined_df['StockholdersEquity']
    )

    combined_df['D/E_Ratio'] = (
        combined_df['Liabilities'] / combined_df['StockholdersEquity']
    )

    combined_df['Current_Ratio'] = (
        combined_df['AssetsCurrent'] / combined_df['LiabilitiesCurrent']
    )

    combined_df['Net_Profit_Margin'] = (
        (combined_df['NetIncomeLoss'] / combined_df['Revenues']) * 100
    )
    
    #combined_df['P/E_Ratio'] = (stock price using yfinance / EPS)


    return combined_df
financial_data = get_financial_data(cik, terms, headers)

# Display the first few rows of the resulting DataFr
financial_data.head(30)

,NetIncomeLoss,InterestExpense,IncomeTaxExpenseBenefit,Depreciation,AmortizationOfIntangibleAssets,Revenues,AssetsCurrent,LiabilitiesCurrent,Liabilities,StockholdersEquity,EarningsPerShareDiluted,EBITDA,ROE,D/E_Ratio,Current_Ratio,Net_Profit_Margin
fy,,,,,,,,,,,,,,,,
2009,-677000000,0.0,-29000000,0.0,53000000,4.212000e+09,2585000000,1574000000,1917000000,2729000000,0.00,-6.530000e+08,-0.248076,0.702455,1.642313,-16.073124
2010,151000000,0.0,-3000000,104000000.0,57000000,3.654000e+09,3032000000,2001000000,2364000000,2564000000,0.00,3.090000e+08,0.058892,0.921997,1.515242,4.132458
2011,96000000,20000000.0,-58000000,102000000.0,43000000,3.589000e+09,2609000000,2120000000,3033000000,3134000000,1.20,2.030000e+08,0.030632,0.967773,1.230660,2.674840
2012,-276000000,29000000.0,-3000000,118000000.0,57000000,4.143000e+09,2325000000,1917000000,2803000000,2267000000,-0.62,-7.500000e+07,-0.121747,1.236436,1.212833,-6.661839
2013,76000000,20000000.0,-1000000,102000000.0,16000000,3.797000e+09,3138000000,2390000000,3294000000,2564000000,1.15,2.130000e+08,0.029641,1.284711,1.312971,2.001580
2014,222000000,31000000.0,50000000,126000000.0,14000000,3.575000e+09,3720000000,2747000000,3080000000,2458000000,1.19,4.430000e+08,0.090317,1.253051,1.354205,6.209790
2015,899000000,28000000.0,-279000000,119000000.0,7000000,4.515000e+09,4354000000,2418000000,3652000000,3396000000,0.03,7.740000e+08,0.264723,1.075383,1.800662,19.911406
2016,442000000,47000000.0,50000000,115000000.0,14000000,4.396000e+09,4354000000,2418000000,3652000000,4060000000,-0.45,6.680000e+08,0.108867,0.899507,1.800662,10.054595
2017,440000000,28000000.0,406000000,115000000.0,9000000,4.845000e+09,6004000000,2491000000,3989000000,3396000000,1.95,9.980000e+08,0.129564,1.174617,2.410277,9.081527
